# Exercise 1: Profile Data

Aim: To work with vertical profile data and make some standard calculations.

- Author: XX YOUR NAME XX
- Purpose: Plot profile data
- Date: YYYY-MM-DD

Four figures should be generated:
- ex1fig1-Lastname-Messfern.png: Showing the temperature and practical salinity
- ex1fig2-Lastname-Messfern.png: Showing the conservative temperature and practical salinity profiles
- ex1fig3-Lastname-Messfern.png: Showing a basic TS profile
- ex1fig4-Lastname-Messfern.png: Showing a TS profile focused on waters denser than 27.
<hr>

In [20]:
# Importing required packages here
import matplotlib.pyplot as plt
import gsw 
import numpy as np
import xarray as xr
import os

from matplotlib.ticker import ScalarFormatter

# For these to work, you need to install CTD-tools (https://gitlab.rrz.uni-hamburg.de/ifmeo/processing/ctd-tools)
from ctd_tools.modules.reader import CnvReader, NetCdfReader
from ctd_tools.modules.writer import NetCdfWriter
from ctd_tools.modules.plotter import CtdPlotter



In [2]:
# Input file paths
cnv_file_with_path = '../data/MSM121_054_1db.cnv'

# Output file paths for netCDF
netcdf_file_with_path = '../data/MSM121_054_1db.nc'
netcdf_file_with_path_edited = '../data/MSM121_054_1db_edited.nc'

# Output file paths for figures
figdir = '../figures/'
if not os.path.exists(figdir):
    os.makedirs(figdir)

## Transforming to xarray

### Convert CNV file to netCDF

In [ ]:
# Using pyCNV
# Data from Seabird sensors typically come in as .cnv files.  
# These files can be converted to netCDF format using the seabird package.
# In this case, we've done the conversion for you.

# converted cnv to netcdf (MSM121_054_1db.cnv --> MSM121_054_1db.nc) on the command line

# Instead of using seabird package on CLI for conversion: 
# do it here within Python, but check before whether the .nc file already exists using os package
#if not os.path.exists(netcdf_file_with_path):
#    data = fCNV(cnv_file_with_path) # read .cnv file
#    cnv2nc(ds, netcdf_file_with_path) # write .nc file

# Using ctd-tools   
# Read CTD data from CNV file
reader = CnvReader(cnv_file_with_path)
dataset = reader.get_data()

# Write dataset with CTD data to netCDF file
writer = NetCdfWriter(dataset)
if not os.path.exists(netcdf_file_with_path):
    writer.write(netcdf_file_with_path)


### Open file and show contents

In [ ]:
# load data
ctd_ds = xr.open_dataset(netcdf_file_with_path)

# print data
print(ctd_ds.info)

## Understanding the data format

The data stored within the netcdf file include measured parameters as `variables`, for example:

- `pressure` is pressure in decibars
- `temperature` is temperature in degrees Celcius
- `salinity` is salinity from the secondary sensor
- `latitude` is the position where the data were take from in degrees North (positive) or south (negative)
- `longitude` is the longitude with degrees east (positive) or west (negative)

The list of `Attributes` include details like the single value for latitude/longitude, or the date the file was created.

In [46]:
# Here we are defining the names of the variables in the netCDF file
# If your netcdf file calls salinity "PSAL", you would change the sal_string to 'PSAL'
sal_string = 'salinity'
temp_string = 'temperature'
pres_string = 'pressure'
lon_string = 'longitude'
lat_string = 'latitude'
SA_string = 'absolute_salinity'
CT_string = 'conservative_temperature'
# You could also define these pairs in a dictionary (https://www.w3schools.com/python/python_dictionaries.asp)

# Plot the temperature and salinity

To plot in Python, we often use the package `matplotlib` which  you've imported above.  This includes a variety of functions which can be used to generate figures in python.

In [ ]:
# To initialise a figure, use matplotlib.plt.figure() which takes inputs within the brackets
# Since matplotlib is long to write, in the import cell above, we've imported it as plt
plt.figure(1, figsize=(12, 8)) 

# In case the figure 1 already exists, clear it using clf()
plt.clf()

# Within the figure window, we'll actually have two separate sets of axes.  These are called 'subplots' in matplotlib
# We'll have two subplots, one next to the other, so we'll have a 1x2 grid of subplots
plt.subplot(1,2,1)

# To plot salinity, we need to reference the salinity variable in the dataset.  This is done using the syntax ctd_ds['PSAL2']
# Here, ctd_ds is the name of the dataset, and 'PSAL2' is the name of the variable.  The variable name is case sensitive.
plt.plot(ctd_ds[sal_string], ctd_ds[pres_string], color='blue')

# Now we'd like to annotate our figure.  We can do this using the plt.xlabel() and plt.ylabel() functions
# These functions take a string as an argument, which is the text that will be displayed on the x and y axes
# An optional second argument can be used to specify the fontsize.  IMPORTANT: choose values to make the text readable.
plt.xlabel('Practical salinity' , fontsize=12)
plt.ylabel('Pressure [dbar]', fontsize=12)

# Since pressure decreases with depth, we want to invert the y-axis so that the surface is at the top of the plot
plt.gca().invert_yaxis()

# Here we specify where exactly the x-ticks should be placed.  We want them at 34, 34.5, and 35
x_ticks = [34, 34.5, 35]
plt.xticks(x_ticks, fontsize=12)

# We don't specify for the y-axis, but we do want the text size to be readable.
plt.yticks(fontsize=12)

# Finally, we can add a title to the plot using plt.title().  This function also takes a string as an argument.
plt.title('Salinity profile', fontsize=12)
plt.grid(True)

# YOUR TURN: Now, let's plot the temperature profile on the right-hand side of the figure

plt.subplot(1,2,2)

# This code saves the figure to a file.  The file type is determined by the extension of the filename.
# Change LastName to your last name
plt.savefig(figdir + 'ex1fig1-LastName-Messfern.png')
plt.show()



# Checklist for figure details to change:

- [ ] Change the shallow limit to be exactly zero
- [ ] Change the gridlines to be dashed (or dotted)
- [ ] Add additional ticks on the x-axis to show more detail
- [ ] Reduce the ticks and gridlines on the y-axis to be every 1000m
- [ ] Change the title to also include the latitude/longitude of the profile
- [ ] Add empty square brackets after Practical Salinity  [ ] to indicate that there are no units
- [ ] Make the figure size about 1/3rd the height of A4 paper, and the text width to fit on A4 paper with margins.
- [ ] Change the font to Times New Roman or another serif font.

## Calculating TEOS-10 parameters

While data are collected as practical salinity and in situ temperature, in Messmethoden, we'll learn that we plot or represent the data by the quantities "Absolute salinity" and "conservative temperature".  These are better tracers of the properties within the ocean.

To convert from practical to absolute salinity and temperature to conservative, we use the TEOS-10 toolbox, sometimes called the "Gibbs Seawater Toolbox".  In Python, you've loaded this as `import gsw`.

https://teos-10.github.io/GSW-Python/conversions.html


In [ ]:
# Calculate absolute salinity
SA_2 = gsw.conversions.SA_from_SP(ctd_ds[sal_string],ctd_ds[pres_string],ctd_ds[lon_string],ctd_ds[lat_string])

# Calculate conservative temperature
CT_2 = gsw.conversions.CT_from_t(SA_2,ctd_ds[temp_string],ctd_ds[pres_string])

type(SA_2)
# Add the new variables to the dataset
# Here we've included scan as a `dimension` for the new variables which is required for the data format netCDF
ctd_ds['absolute_salinity'] = (("time"), SA_2.values)
ctd_ds['absolute_salinity'].attrs['long_name'] = 'Absolute Salinity [g/kg]'
ctd_ds['conservative_temperature'] = (("time"), CT_2.values)
ctd_ds['conservative_temperature'].attrs['long_name'] = 'Conservative Temperature [°C]'
# Note in the lines above, to add a new variable to the dataset, we use the syntax ctd_ds['variable_name'] = (dimensions, data)
# Since the dimensions of the previous variables were (time), we need to include these dimensions for the new variables as well
# The .values attribute is used to extract the data from the xarray DataArray object

# Check the data.  Note that the new variables are now in the dataset.
print(ctd_ds.info)

You can also calculate and add the field to the data structure in a single line

In [41]:
# Calculate depth
#ctd_ds['DEPTH'] = gsw.z_from_p(ctd_ds['PRES'], ctd_ds['LATITUDE']) # convert pressure to depth


## Plot absolute and conservative temperature profiles

In [ ]:
# Your code here

## Plot the absolute salinity and conservative temperature profiles against depth in meters.

# Change Lastname to your last name
plt.savefig(figdir + 'ex1fig2-LastName-Messfern.png')
plt.show()

### Saving xarray to netCDF file

Since we've manipulated the data, and may want to use the new variables "Absolute salinity" and "Conservative temperature" again later, let's save the new data file.


In [43]:
ctd_ds.to_netcdf(netcdf_file_with_path_edited)


## T-S Diagram

A T-S diagram has salinity on the x-axis and temperature on the y-axis, and contours of density (sigma_0) added.

In [ ]:
#https://stackoverflow.com/questions/41634179/matplotlib-contour-lines-are-not-closing-up

# On a TS diagram, the X axis is salinity and the Y axis is temperature.  
# As a helpful reminder, we can create variables X and Y.
x = ctd_ds[SA_string]
y = ctd_ds[CT_string]


# A typical TS diagram also has contours of potential density.  We can add these using the gsw package.
# meshgrid creates a grid of points for the contour plot.  The first two arguments are the x and y values,
X, Y = np.meshgrid(np.linspace(min(x), max(x), 100), np.linspace(min(y), max(y), 100))
# The third argument is the potential density, which is calculated using the gsw.density.sigma0() function
Z = gsw.density.sigma0(X, Y)

# Now we can plot the data.  We'll use the plt.plot() function for the curve with temperature and salinity, which takes the x and y values as arguments.
plt.figure(figsize=(3, 8)) 
plt.plot(x,y, color='green', marker='o', markersize='6')
plt.xlabel('Absolute Salinity', fontsize=14)
plt.ylabel(' Temperature [°C]', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(1.5,14.4)
plt.title('TS diagram', fontsize=18)

# Then we add contours using the plt.contour() function.  This function takes the X, Y, and Z values as arguments.
CS = plt.contour(X, Y, Z, 10, colors='grey', zorder=2)
plt.clabel(CS, inline=True, fontsize=12)

plt.grid(True)
# Change Lastname to your last name
plt.savefig(figdir + 'ex1fig3-LastName-Messfern.png')
plt.show()

In [ ]:
# Repeat the plot here, but this time, plot the contours and then the data points.  This will make the data points more visible.

# Try removing the grid lines

# Change the aspect ratio of the plot to make it square

# Change the color of the line to red

# Change the axes limits to focus on the data denser than 27.00 kg/m^3

# Save the figure as a .png file in the figures directory
# Name the figure ex1fig4-LastName-Messfern.png